In [13]:
import pandas as pd
import numpy as np

---

- df_외감 : 2022 신용등급
- df : 우리가 가진 데이터 프레임

In [14]:
df_외감=pd.read_excel('./Dataset/외감_dp(등급).xls')
df_외감.columns = ['KIS','Stock','회사명','신용평점']

In [15]:
df=pd.read_csv('../외감기업/Dataset/외감raw3.csv',encoding='euc-kr')
df['Year']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)
df=df[df['Year']==2022]
df['자산(*)(IFRS연결)(천원)'] = df['자산(*)(IFRS연결)(천원)'] * 1000
df['log자산총계'] = np.log(df['자산(*)(IFRS연결)(천원)'])
df['잉여현금흐름(FCF)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']-df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['전체현금흐름(천원)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']+df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']+df['재무활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산(*)(IFRS연결)(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산(*)(IFRS연결)(천원)']
df=df[['회사명', '거래소코드','유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]

c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
# df.to_csv('등급화_외감.csv')

---

- df_등급화 : merge (회사명을 기준으로)

In [17]:
df_등급화=pd.merge(df,df_외감,how='inner',on='회사명')

In [18]:
df_등급화['신용평점'].value_counts()

BBB    3732
BB     2863
B      2687
A      2053
CC     1563
CCC     692
C       554
AA      406
D       142
AAA       1
Name: 신용평점, dtype: int64

In [19]:
df_등급화.isna().sum()

회사명                0
거래소코드              0
유동자산회전률            3
총자산대비잉여현금흐름        3
자기자본구성비율           2
log자산총계            2
자기자본회전률            3
순운전자본회전률           3
자기자본증가율            2
총자본증가율             2
총자산대비현금흐름          3
총자본투자효율            2
KIS                0
Stock          14540
신용평점               0
dtype: int64

In [20]:
df_등급화=df_등급화.drop('Stock',axis=1)

In [21]:
df_등급화=df_등급화.dropna()
df_등급화 = df_등급화[(df_등급화 != 0).all(axis=1)]

In [22]:
df_등급화 = df_등급화.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df_등급화 = df_등급화.dropna()  # NaN이 있는 행 삭제

In [23]:
import numpy as np

# O-Score의 계수
coefficients = {
    '유동자산회전률':0.47765086,
    '총자산대비잉여현금흐름':0.53785436,
    '자기자본구성비율':0.49360789,
    'log자산총계':0.94238478,
    '자기자본회전률':-0.08646556,
    '순운전자본회전률':-0.98107367,
    '자기자본증가율':0.19544705,
    '총자본증가율':0.44094258,
    '총자산대비현금흐름':-0.37036301,
    '총자본투자효율':0.26458857,

    'constant': 0.90495308
}

# O-Score 계산
df_등급화['dp_score'] = coefficients['constant']
df_등급화['dp_score'] += coefficients['유동자산회전률'] * df_등급화['유동자산회전률']
df_등급화['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df_등급화['총자산대비잉여현금흐름']
df_등급화['dp_score'] += coefficients['자기자본구성비율'] * df_등급화['자기자본구성비율']
df_등급화['dp_score'] += coefficients['log자산총계'] * df_등급화['log자산총계']
df_등급화['dp_score'] += coefficients['자기자본회전률'] * df_등급화['자기자본회전률']
df_등급화['dp_score'] += coefficients['순운전자본회전률'] * df_등급화['순운전자본회전률']
df_등급화['dp_score'] += coefficients['자기자본증가율'] * df_등급화['자기자본증가율']
df_등급화['dp_score'] += coefficients['총자본증가율'] * df_등급화['총자본증가율']
df_등급화['dp_score'] += coefficients['총자산대비현금흐름'] * df_등급화['총자산대비현금흐름']
df_등급화['dp_score'] += coefficients['총자본투자효율'] * df_등급화['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [24]:
df_등급화[['dp_score','신용평점']].groupby('신용평점').mean()

,dp_score
신용평점,
A,76.479800
AA,77.900646
AAA,101.204536
B,59.562975
BB,57.882489
BBB,70.520457
C,49.292018
CC,54.179345
CCC,57.872817
